In [1]:
%pip install requests beautifulsoup4 pandas re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re

# Target URL
url = 'http://darksouls.wikidot.com/classes'

browser = "chrome"

USER_AGENTS = {
    "chrome": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "safari": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15"
}

headers = {
    "User-Agent": USER_AGENTS[browser]
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
table = soup.find('table', {'class': 'wiki-content-table'})
rows = table.find_all('tr')

# Step 3: Extract headers
headers = [th.get_text(strip=True).split('\n')[0] for th in rows[0].find_all('th')][1:]




In [4]:
# Step 4: Extract each row
data = []
for row in rows[1:]:
    cols = row.find_all('td')
    if not cols:
        continue

    class_name = cols[0].get_text(strip=True)
    values = [col.get_text(strip=True) for col in cols[1:]]

    attr = {}
    for key, val in zip(headers, values):
        # Remove anything in parentheses and convert to int
        clean_val = re.sub(r"\s*\(.*?\)", "", val).strip()
        try:
            attr[key if key != "Total Points (At Starting Level)" else "Total Points"] = int(clean_val)
        except ValueError:
            attr[key] = clean_val  # fallback if still not clean

    data.append({
        "class": class_name,
        "attributes": attr
    })

In [5]:
# Step 5: Save to JSON
with open('dark_souls_classes_clean.json', 'w') as f:
    json.dump(data, f, indent=2)